In [1]:
import sys, os
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from scipy.stats import ttest_ind_from_stats
from scipy.spatial import distance
import random
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
np.random.seed(0)

In [2]:
sys.argv[1] = 'digits-embedding.csv'
sys.argv[2] = 10

data = pd.read_csv(sys.argv[1], header=None)
K = int(sys.argv[2])

In [9]:
N = len(data)
centroid_idx = np.random.randint(0, N, size=K)
print 'N:', N

N: 20000


In [10]:
# Initial the centroid
centroid = []
for i in centroid_idx:
    centroid.append((data.loc[i][2], data.loc[i][3]))

In [11]:
# print centroid
label = [0 for i in range(len(data))]
count = 0
while True:
    update = 0
    x = [0.0 for i in range(K)]
    y = [0.0 for i in range(K)]
    label_count = [0 for i in range(K)]
    for index, row in data.iterrows():
        coor = (row[2], row[3])
        candidate_label = []
        for c in centroid:
            candidate_label.append(distance.euclidean(coor, c))
        # Get the label of the current point
        l = candidate_label.index(min(candidate_label))
        x[l] += row[2]
        y[l] += row[3]
        label_count[l] += 1
        
        # Check if there's an update in each round
        if l != label[index]:
            label[index] = l
            update += 1
            
    count += 1
    
    if count == 50 or update == 0:
        break
    
    # Update the centroid if they does not meet the stop criterion
    for i in range(len(centroid)):
        centroid[i] = (1.0 * x[i]/label_count[i], 1.0 * y[i]/label_count[i])
    
    print count, update

1 18707
2 4201
3 2665
4 1833
5 1404
6 1349
7 1190
8 1246
9 1332
10 1060
11 758
12 502
13 336
14 225
15 191
16 192
17 150
18 88
19 61
20 53
21 41
22 40
23 26
24 27
25 36
26 36
27 37
28 37
29 31
30 41
31 43
32 37
33 26
34 21
35 23
36 11
37 6
38 5
39 3
40 4
41 8
42 7
43 3
44 3
45 6


In [12]:
WC_SSD = 0
S = []
P = [[0 for i in range(K)] for j in range(10)]
H_C = [0 for i in range(K)]
H_G = [0 for i in range(10)]

for index, row in data.iterrows():
    print index
    # Update the WC_SSD
    WC_SSD += distance.euclidean((row[2], row[3]), centroid[label[index]])
    A = [0 for i in range(N) if (row[1] == label[i]) else 1]
    B = [0 for i in range(N) if (row[1] != label[i]) else 1]
#     for i, r in data.iterrows():
#         if i != index and label[i] == label[index]:
    #A[i] = distance.euclidean((row[2], row[3]), (r[2], r[3]))
#         elif i != index and label[i] != label[index]:
    #B[i] = distance.euclidean((row[2], row[3]), (r[2], r[3]))
    
    # Update the information matrix
    P[label[index]][int(row[1])] += 1
    H_C[label[index]] += 1
    H_G[int(row[1])] += 1
    
    SC_A = 1.0 * np.sum(A)/np.count_nonzero(A)
    SC_B = 1.0 * np.sum(B)/np.count_nonzero(A)
    
    S.append((SC_B-SC_A)/max(SC_A, SC_B))

print 'WC_SSD:', WC_SSD
print 'SC:', sum(S)/len(S)

SyntaxError: invalid syntax (<ipython-input-12-d490f88f52bc>, line 11)

In [ ]:
# Get the NMI
H_C_Entropy = 0
H_G_Entropy = 0
MI = 0
for i in range(K):
    H_C_Entropy -= (1.0 * H_C[i]/N) * np.log(1.0 * H_C[i]/N)
for i in range(10):
    H_G_Entropy -= (1.0 * H_G[i]/N) * np.log(1.0 * H_G[i]/N)
for i in range(K):
    for j in range(10):
        if P[i][j] != 0:
            MI += (1.0 * p[i][j]/N) * np.log(N * P[i][j]/(H_C[i] * H_G[i]))
print 'NMI:', MI